In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

torch.manual_seed(0)
batch_size = 64
num_epochs = 10
learning_rate = 0.01
decay_rate = 0.1
num_classes = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform)

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2)

testset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 4 * 4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, num_classes)
    
    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=decay_rate)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(trainloader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 200 ==199: 
            print('[%d, %5d] loss: %.3f' % (epoch +1,i+1,running_loss/200))
            running_loss=0.0
            
    scheduler.step() 

print('Finished Training')


[1,   200] loss: 1.624
[1,   400] loss: 0.244
[1,   600] loss: 0.139
[1,   800] loss: 0.110
[2,   200] loss: 0.085
[2,   400] loss: 0.065
[2,   600] loss: 0.065
[2,   800] loss: 0.063
[3,   200] loss: 0.046
[3,   400] loss: 0.049
[3,   600] loss: 0.051
[3,   800] loss: 0.045
[4,   200] loss: 0.035
[4,   400] loss: 0.041
[4,   600] loss: 0.035
[4,   800] loss: 0.035
[5,   200] loss: 0.028
[5,   400] loss: 0.034
[5,   600] loss: 0.029
[5,   800] loss: 0.031
[6,   200] loss: 0.021
[6,   400] loss: 0.019
[6,   600] loss: 0.016
[6,   800] loss: 0.015
[7,   200] loss: 0.014
[7,   400] loss: 0.014
[7,   600] loss: 0.014
[7,   800] loss: 0.013
[8,   200] loss: 0.012
[8,   400] loss: 0.013
[8,   600] loss: 0.010
[8,   800] loss: 0.015
[9,   200] loss: 0.010
[9,   400] loss: 0.011
[9,   600] loss: 0.013
[9,   800] loss: 0.010
[10,   200] loss: 0.012
[10,   400] loss: 0.013
[10,   600] loss: 0.012
[10,   800] loss: 0.008
Finished Training
